In [ ]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import pandas as pd

wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(['title', 'author', 'genre', 'story', 'score', 'platform'])

raw = requests.get("https://comic.naver.com/webtoon/finish.nhn?view=list&order=Update",
                   headers = {"User-Agent" : "Mozilla/5.0"})
html = BeautifulSoup(raw.text, 'html.parser')
webtoons = html.select("tbody > tr")

for webtoon in webtoons:
    title = webtoon.select_one("td:nth-of-type(1) a")
    url = title.attrs["href"]

    score = webtoon.select_one("div.rating_type > strong").text
    author = webtoon.select_one("td:nth-of-type(3) a").text


    each_raw = requests.get("https://comic.naver.com" + url,
                            headers = {"User-Agent" : "Mozilla/5.0"})
    each_html = BeautifulSoup(each_raw.text, 'html.parser')

    genre = each_html.select_one("p.detail_info span.genre").text
    storysum = each_html.select_one("div.detail p").text


    print("TITLE : " +title.text)
    print("-"*30)
    print("SCORE : " + score)
    print("AUTHOR : " + author)
    print("GENRE : " +genre)
    print("STORY : " +storysum)
    print("="*50)

    sheet.append([title.text, author, genre, storysum,  score])

wb.save("NAVER_FINISH.xlsx")

In [ ]:
# 작업전 생성된 XLSX 파일을 CSV 파일로 변환 후 작업할 것

df = pd.read_csv('NAVER_FINISH.csv', encoding='utf-8')

In [ ]:
df

In [ ]:
import re

def preprocess_sentence_kr(w):
  w = w.strip()
  w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
  w = w.strip()
  return w

In [ ]:
df['story'] = [preprocess_sentence_kr(l) for l in df['story']]

In [ ]:
df.to_csv('NAVER_FINISH1.csv', encoding='euc-kr')